In [2]:
import sympy as sp
from scipy.optimize import minimize
import numpy as np
from scipy.misc import derivative

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# DATA-585 Assignment 2

## Part I
### 1. Finding first derivative and if stationary points can be found analytically?

In [3]:
x = sp.Symbol('x')
eqn = sp.exp(x) + x**12/12 + x**6/6 + x
derivative_x = sp.diff(eqn, x)
print("g(x) = f'(x) =", derivative_x)

g(x) = f'(x) = x**11 + x**5 + exp(x) + 1


You cannot find stationary points analytically for the derivative because of the exponential term and high-order polynomial. As such, estimation techniques would need to be used to find the stationary points.   

### 2. Finding second derivative and if f is convex?

In [4]:
sec_derivative = sp.diff(derivative_x, x)
print("H(x) = f''(x) =", sec_derivative)

H(x) = f''(x) = 11*x**10 + 5*x**4 + exp(x)


Because H(x) >= 0 for all x, f is convex. 

### 3. Finding x which minimizes f(x)

In [5]:
def f(x):
    return (1/12)*x**12 + (1/6)*x**6 + x + np.exp(x)

x0 = [0]

result = minimize(f, x0)
print(result)

print("\nx value that minimizes the function:", result.x)
print("Minimum value of the function:", result.fun)

      fun: -0.3958010277871996
 hess_inv: array([[0.08829298]])
      jac: array([2.80886889e-06])
  message: 'Optimization terminated successfully.'
     nfev: 14
      nit: 5
     njev: 7
   status: 0
  success: True
        x: array([-0.95393675])

x value that minimizes the function: [-0.95393675]
Minimum value of the function: -0.3958010277871996


### 4. Newton's Method for Optimization

In [6]:
def f_prime(x):
    return x**11 + x**5 + 1 + np.exp(x)

def f_prime_prime(x):
    return (11)*x**10 + (5)*x**4 + np.exp(x)

x0 = 1
n = 15

for i in range(n):
    x = x0 - f_prime(x0)/f_prime_prime(x0)
    print(f"{i+1}: {x}")
    x0 = x

1: 0.694508188258762
2: -0.2270877325345032
3: -2.44426149039445
4: -2.2215019268305984
5: -2.0186691336289804
6: -1.833777392507754
7: -1.6649552546536384
8: -1.5104634742402345
9: -1.3688476977630728
10: -1.239495346053174
11: -1.1242382120217118
12: -1.0306779931062928
13: -0.9731765702068925
14: -0.9553191468040753
15: -0.9539444224713599


### 5. Gradient Descent with Armijo Backtracking

In [7]:
def f(x):
    return (1/12)*x**12 + (1/6)*x**6 + x + np.exp(x)
    
def f_prime(x):
    return x**11 + x**5 + 1 + np.exp(x)
    
x0 = 1
n = 15

for i in range(n):
    t_n = 1
    # Armijo backtracking 
    while [f(x0 - t_n*f_prime(x0))] > [f(x0) - 0.5*t_n*(f_prime(x0))**2]:
        t_n = t_n/2
    x = x0 - t_n*f_prime(x0)
    print(f"{i+1}: {x}")
    x0 = x

1: 0.2852147714426194
2: -0.8807532549752439
3: -0.9205706188822561
4: -0.9414950741591653
5: -0.9499362176638271
6: -0.952740237579373
7: -0.9535881979257744
8: -0.9538361524907042
9: -0.9539079058230353
10: -0.9539286062017607
11: -0.9539345728122027
12: -0.9539362921666185
13: -0.9539367875836582
14: -0.9539369303307255
15: -0.9539369714609212


## Part II

### 1. Finding gradient

In [8]:
x1, x2 = sp.symbols('x1 x2')
f = sp.exp(x1) + 3*x1**2 + 5*x2**2 - 4*x1*x2 
gradient = sp.Matrix([sp.diff(f, var) for var in (x1, x2)])
print(gradient)

Matrix([[6*x1 - 4*x2 + exp(x1)], [-4*x1 + 10*x2]])


## 2. Finding Hessian

In [9]:
f_xx = sp.diff(f, x1, x1)  # Second derivative with respect to x1
f_xy = sp.diff(f, x1, x2)  # Mixed partial derivative x1 then x2
f_yx = sp.diff(f, x2, x1)  # Mixed partial derivative x2 then x1
f_yy = sp.diff(f, x2, x2)  # Second derivative with respect to x2

hessian = sp.Matrix([[f_xx, f_xy], [f_yx, f_yy]])
print(hessian)

eigenval = hessian.eigenvals()
print(f"Eigen values of the hessian: {eigenval}")

Matrix([[exp(x1) + 6, -4], [-4, 10]])
Eigen values of the hessian: {-sqrt(exp(2*x1) - 8*exp(x1) + 80)/2 + exp(x1)/2 + 8: 1, sqrt(exp(2*x1) - 8*exp(x1) + 80)/2 + exp(x1)/2 + 8: 1}


Because the eigen values of the hessian are both non-negative (the first term has a negative square root but exp(x1)/2 + 8 makes sure its positive for all x1) and the 2nd eigen value has no negative values (outside the sqrt and sqrt has to always be positive for real numbers), thus the eigen values are always positive for all x1. Because the eigen values of the hessian are non-negative, f''(x) is >= 0 for all (x1, x2), and so f is convex. Because f is convex, any stationary point is a global minimizer.  

### 3. Find optimal solution

In [10]:
def f(x):
    return np.exp(x[0]) + 3*x[0]**2 + 5*x[1]**2 - 4*x[0]*x[1] 

# Initial guess
x0 = np.array([0, 0])

result = minimize(f, x0, method='BFGS')
print(result)

print("\nOptimal solution:", result.x)
print("Minimum value of the function:", result.fun)

      fun: 0.906371315845204
 hess_inv: array([[0.18880106, 0.07622719],
       [0.07622719, 0.13049609]])
      jac: array([-7.34627247e-06,  3.47197056e-06])
  message: 'Optimization terminated successfully.'
     nfev: 18
      nit: 3
     njev: 6
   status: 0
  success: True
        x: array([-0.18827173, -0.07530835])

Optimal solution: [-0.18827173 -0.07530835]
Minimum value of the function: 0.906371315845204


### 4. Newton's Method

In [11]:
def f(x):
    return np.exp(x[0]) + 3*x[0]**2 + 5*x[1]**2 - 4*x[0]*x[1]

def gradient(x):
    return np.array([np.exp(x[0]) + 6*x[0] - 4*x[1], 10*x[1] - 4*x[0]])

def hessian(x):
    return np.array([[np.exp(x[0]) + 6, -4], [-4, 10]])

x0 = np.array([8, 1])
f_x = 100
i = 1

while np.abs(f_x - 0.906371315845204) > 1e-6:
    hess_inv = np.linalg.inv(hessian(x0))
    x = x0 - hess_inv.dot(gradient(x0))
    f_x = f(x)
    x0 = x
    print(f"{i}: \n{x}")
    print(f"{f_x}\n")
    i += 1
    
    


1: 
[6.98968298 2.79587319]
1192.859804544672

2: 
[5.9654995 2.3861998]
468.03946481578765

3: 
[4.910068  1.9640272]
188.68792692258776

4: 
[3.78722282 1.51488913]
75.68838777990014

5: 
[2.53453676 1.0138147 ]
26.74311619345058

6: 
[1.13760975 0.4550439 ]
5.96644658227158

7: 
[0.05708595 0.02283438]
1.0659161753875501

8: 
[-0.18288213 -0.07315285]
0.9064472415660323

9: 
[-0.18826827 -0.07530731]
0.9063713158550559



It took 9 iterations for the function value to be within 10^-6 of the optimal value from part 3 using Newton's Method.

### 5. Gradient descent with Armijo Backtracking

In [31]:
def f(x):
    return np.exp(x[0]) + 3*x[0]**2 + 5*x[1]**2 - 4*x[0]*x[1]

def gradient(x):
    return np.array([np.exp(x[0]) + 6*x[0] - 4*x[1], 10*x[1] - 4*x[0]])

x0 = np.array([8, 1])
f_x = 100
i = 1

while np.abs(f_x - 0.906371315845204) > 1e-6:
    t_n = 1 
    while (f(x0 - t_n * gradient(x0)) > f(x0) - 0.5 * t_n * (gradient(x0))**2).all():
        t_n = t_n/2
    x = x0 - t_n * gradient(x0)
    f_x = f(x)
    print(f"{i}: \n{x}")
    print(f"{f_x}\n")
    x0 = x
    i += 1

1: 
[-15.63248427   1.171875  ]
813.2674189822978

2: 
[-3.32218359 -8.10921089]
254.18214210213694

3: 
[-4.88966059  0.36621093]
79.56700672771643

4: 
[-1.04025018 -2.53638303]
25.21203024269087

5: 
[-1.57242484  0.11397067]
8.406888750519727

6: 
[-0.36206354 -0.81470509]
3.228330008534434

7: 
[-0.58489819  0.0226445 ]
1.6390230536301567

8: 
[-0.20454762 -0.29811022]
1.140972693798533

9: 
[-0.30206901 -0.02774625]
0.98334824152265

10: 
[-0.24193513 -0.0859221 ]
0.9144676673275105

11: 
[-0.20158322 -0.09948704]
0.9086116161414591

12: 
[-0.20231877 -0.07591985]
0.9070122336322594

13: 
[-0.19064393 -0.08217942]
0.9065613808271139

14: 
[-0.19205404 -0.07477711]
0.9064296296769936

15: 
[-0.18855985 -0.07733274]
0.9063897522735437

16: 
[-0.1893252  -0.07494674]
0.9063772914066194

17: 
[-0.18824434 -0.07592592]
0.9063732906964491

18: 
[-0.18857558 -0.07514069]
0.9063719781915331



It took 18 iterations for the function value to be within 10^-6 of the optimal value from part 3 using gradient descent with Armijo backtracking.